In [23]:
%matplotlib inline
import re
import itertools
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
from matplotlib.colors import rgb2hex
from matplotlib.patches import Polygon
import matplotlib.ticker as ticker

In [13]:
ridership = pd.read_csv('CTA_-_Ridership_-__L__Station_Entries_-_Daily_Totals.csv')
stops = pd.read_csv('CTA_-_System_Information_-_List_of__L__Stops.csv')

In [14]:
ridership.shape

(879576, 5)

In [15]:
ridership.columns

Index(['station_id', 'stationname', 'date', 'daytype', 'rides'], dtype='object')

In [16]:
ridership[:5]

,station_id,stationname,date,daytype,rides
0,40010,Austin-Forest Park,01/01/2001,U,290
1,40020,Harlem-Lake,01/01/2001,U,633
2,40030,Pulaski-Lake,01/01/2001,U,483
3,40040,Quincy/Wells,01/01/2001,U,374
4,40050,Davis,01/01/2001,U,804


In [17]:
stops[:5]

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location
0,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
1,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
4,30213,N,35-Bronzeville-IIT (Harlem-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,False,False,False,False,False,False,"(41.831677, -87.625826)"


In [18]:
stops.loc[stops['MAP_ID'] == 40830]

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location
0,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
1,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"


In [19]:
ridership.loc[ridership['station_id'] == 40830]

,station_id,stationname,date,daytype,rides
79,40830,18th,01/01/2001,U,0
220,40830,18th,01/02/2001,W,813
361,40830,18th,01/03/2001,W,881
502,40830,18th,01/04/2001,W,884
643,40830,18th,01/05/2001,W,925
784,40830,18th,01/06/2001,A,2
925,40830,18th,01/07/2001,U,0
1066,40830,18th,01/08/2001,W,981
1207,40830,18th,01/09/2001,W,979
1348,40830,18th,01/10/2001,W,987


In [22]:
#just want 2017 ridership
type(ridership.date[1])

str

In [27]:
#add a new column for year, then regex the year out of date columns
ridership["year"] = ""
ridership.year = ridership.date.str.extract(r'./+(\d{4})', expand=False)
ridership_17 = ridership['year'] == 2017

In [40]:
#aggregate stops by map_id, join with ridership on station_id?
#or get the shapefile for L trains, plot in python, match each station to the number of rides
#plot basic info i.e. ridership/station?